In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import datetime
print(tf.__version__)

In [ ]:
import keras.backend as K
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
model = tf.keras.models.load_model('../input/plant-pathology-01/mobilenet_plant_v1.h5',custom_objects={'f1': f1})
#'somefile', custom_objects={'x': x}

In [ ]:
names = os.listdir('../input/plant-pathology-2021-fgvc8/test_images')

In [ ]:
IMSIZE = 128
def _parse(name):
    image_string = tf.io.read_file('../input/plant-pathology-2021-fgvc8/test_images/' + name)
    image_decoded = tf.image.decode_jpeg(image_string)
    imgs = tf.image.resize(image_decoded, [IMSIZE, IMSIZE])
    return imgs/255



dataset = tf.data.Dataset.from_tensor_slices((tf.constant(names)))\
                               .map(_parse, num_parallel_calls=tf.data.AUTOTUNE)\
                               .batch(32)\
                               .prefetch(tf.data.AUTOTUNE)

In [ ]:
y_pred = model.predict(dataset, verbose=1)

In [ ]:
label_names=['complex','frog_eye_leaf_spot','frog_eye_leaf_spot complex','healthy','powdery_mildew','powdery_mildew complex','rust','rust complex','rust frog_eye_leaf_spot','scab','scab frog_eye_leaf_spot','scab frog_eye_leaf_spot complex']
y = np.around(y_pred)
i = 0
labels = []
for i in range(len(y)):
    vec = str()
    for j in range(len(label_names)):
        if(y[i][j]==1):
            vec = vec + label_names[j] + " "
    labels.append(vec)

In [ ]:
df = pd.DataFrame({'image':names, 'labels':labels})

In [ ]:
df.head()

In [ ]:
df.to_csv('./submission.csv', index=False)

In [ ]:
!cat "/kaggle/working/submission.csv"